In [2]:
from huggingface_hub import QuestionAnsweringInput
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from transformers.utils import logging
logging.get_logger().setLevel("INFO")

model = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B'
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)


/home/kavach/Dev/anaconda3/envs/ALLM-Rec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
loading file tokenizer.model from cache at None
loading file tokenizer.json from cache at /home/kavach/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-7B/snapshots/14dd1130311655b43c3ce41dd505f70f6ca89845/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/kavach/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-7B/snapshots/14dd1130311655b43c3ce41dd505f70f6ca89845/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
l

In [3]:
# Gernerate text

inputText = 'What upon a Time, in India'
input_ids=tokenizer(inputText, return_tensors="pt").input_ids

output_ids = model.generate(input_ids, max_length=100)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


What upon a Time, in India, a mathematician had a method to solve equations, but that method was not known to the Western World until 999 years later. The mathematician was a contemporary of Ramanujan, and his work was considered to be a significant contribution to the field of mathematics, but it was unknown in the West until much later. The mathematician's name is 7 letters long, starts with a 'C', and ends with a 'C'.


In [3]:
# Question Answering
context ='''
The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France.
It was designed by Gustave Eiffel and completed in 1889. The tower is 330 meters tall and is one of the most recognizable structures in the world.
'''
question = 'Who designed the Eiffel Tower ?'
inputText = f'Context:{context}\n\nQuestion:{question}\n\nAnswer: '

# Tokenise Input
input_ids= tokenizer(inputText, return_tensors='pt').input_ids

# Tokenise Output
output_ids = model.generate(input_ids, max_length=100)
output_text= tokenizer.decode(output_ids[0], skip_special_tokens=True)

print('Generated Answer: ', output_text)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Answer:  Context:
The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France.
It was designed by Gustave Eiffel and completed in 1889. The tower is 330 meters tall and is one of the most recognizable structures in the world.


Question:Who designed the Eiffel Tower ?

Answer:  Gustave Eiffel

But, I need to write a sentence that includes both the question and the answer


In [8]:
question = 'Who built  Qutub Minar ?'
inputText = f'Question:{question}\n\nAnswer:'

input_ids= tokenizer(inputText, return_tensors='pt').input_ids

output_ids = model.generate(input_ids, max_length=1000)
output_text= tokenizer.decode(output_ids[0], skip_special_tokens=True)

print('Generated Answer: ', output_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Generated Answer:  Question:Who built  Qutub Minar ?

Answer: The Qutub Minar was built by the Mughal Emperor Suleiman the Magnificent.

But I need to find out the exact date when it was constructed.

I remember it was built around the 16th century, but I'm not sure if it's 1590, 1594, or 1600.

I think it's 1594, but I'm not certain.

Maybe I should check a reliable source or use some clues from the structure or other historical records.

Wait, the Mughal Empire was active from the 16th to the 18th century, so 1594 falls within that period.

I've heard that the Mughal Emperor Suleiman the Magnificent was known for his architectural achievements, so he might have been the builder.

But I need to confirm the exact year.

Perhaps I can recall any specific events or anniversaries related to the construction of Qutub Minar.

Alternatively, I can think about other Mughal buildings and their construction timelines to estimate.

I think the Qutub Minar was built during the reign of Suleiman I

In [12]:
# Story Generation
def generate_story(prompt):
    # Tokenize the prompt (user's input)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Generate text based on the input prompt
    output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=150, temperature=0.7, top_p=0.9)

    # Decode the generated text back to human-readable text
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Return the generated text (story or dialogue)
    return output_text

# Example usage: Prompt for generating a creative story
print(generate_story("Once upon a time, in a world full of mystery"))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Once upon a time, in a world full of mystery, there are two sets of numbers: set \( A \) and set \( B \). Set \( A \) contains the numbers \( 9999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999


In [11]:
def summarize_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate the summary using the model
    summary_ids = model.generate(inputs['input_ids'], max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary and return it
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage: Summarizing a long document
document = """
In the rapidly evolving world of artificial intelligence, the need for advanced algorithms that can
process vast amounts of data has never been greater. The advent of neural networks and deep learning
has paved the way for significant breakthroughs in fields such as natural language processing, computer vision,
and robotics. These systems have been able to outperform traditional methods in many areas, but challenges
remain in terms of interpretability, ethics, and ensuring that AI systems are used in ways that benefit society.
"""
summary = summarize_text(document)
print("Summary:", summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Summary: 
In the rapidly evolving world of artificial intelligence, the need for advanced algorithms that can
process vast amounts of data has never been greater. The advent of neural networks and deep learning
has paved the way for significant breakthroughs in fields such as natural language processing, computer vision,
and robotics. These systems have been able to outperform traditional methods in many areas, but challenges
remain in terms of interpretability, ethics, and ensuring that AI systems are used in ways that benefit society.
As AI continues to advance, it's crucial to address these challenges to ensure that the technology can be
applied effectively and responsibly in the real world.

Let me know if you'd like me to expand on any part of this prompt, or if you'd like me to write a different type of response.

Okay, so I need to write a response to this prompt about the challenges in AI, specifically focusing on interpretability, ethics, and societal impact. I should make sur

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# masti